Instalamos e importamos librerías

In [ ]:
# En algunos casos, puede ser necesario cambiar % por !, dependiendo de la versión de python en uso

%pip install pandas 
%pip install statsmodels 
%pip install linearmodels 
%pip install matplotlib 
%pip install numpy

In [ ]:
# Ahora las importamos

import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np

Importamos los datos

In [50]:
df = pd.read_stata('../Bases de Datos .dta/autor-jole-2003.dta')
df

,state,year,ue,cpsths,tempue,hrwage,wkwage,lnhrw,lnwkw,lnftw,...,annemp,totemp,bls_ue,region,statelf,natlf,reglf,natue,regue,union
0,11.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,361500.0,78714400.0,NaN,1,NaN,0.0,0.0,0.000000,0.000000,NaN
1,11.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,356900.0,77387696.0,NaN,1,NaN,0.0,0.0,0.000000,0.000000,20.700001
2,11.0,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,375300.0,79863504.0,NaN,1,NaN,0.0,0.0,0.000000,0.000000,NaN
3,11.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,387800.0,82970000.0,NaN,1,NaN,0.0,0.0,0.000000,0.000000,NaN
4,11.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,405600.0,87369200.0,0.062,1,432409.3750,92354648.0,5544384.5,0.060438,0.057785,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,95.0,93.0,0.043933,0.003606,0.000000,13.194592,520.331238,2.437165,6.041272,6.222130,...,538800.0,111060304.0,0.043,9,563009.3750,118561184.0,17981786.0,0.068920,0.088055,NaN
1196,95.0,94.0,0.060942,0.003455,0.000000,13.323984,520.674988,2.436631,6.012619,6.188493,...,536200.0,114176896.0,0.061,9,571033.0000,120860816.0,18058438.0,0.060753,0.079533,NaN
1197,95.0,95.0,0.057788,0.002229,0.000000,13.050425,507.867493,2.417453,5.979689,6.177753,...,532800.0,117092096.0,0.059,9,566206.1875,123315104.0,18319228.0,0.055675,0.072979,NaN
1198,95.0,96.0,0.058244,0.004537,0.001337,13.339598,509.207947,2.430573,5.967892,6.180829,...,530700.0,119550296.0,0.064,9,566987.1875,125680008.0,18735586.0,0.053730,0.069797,NaN


In [51]:
df['state']

0       11.0
1       11.0
2       11.0
3       11.0
4       11.0
        ... 
1195    95.0
1196    95.0
1197    95.0
1198    95.0
1199    95.0
Name: state, Length: 1200, dtype: float32

Creamos variables

In [52]:
# Log total employment: from BLS employment & earnings
df['lnemp'] = np.log(df['annemp'])  
# Non-business-service sector employment from CBP
df['nonemp'] = df['stateemp'] - df['svcemp']
df['lnnon'] = np.log(df['nonemp'])
df['svcfrac'] = df['svcemp'] / df['nonemp']
#Total business services employment from CBP
df['bizemp'] = df['svcemp'] + df['peremp']
df['lnbiz'] = np.log(df['bizemp'])
# Time trends
df['t'] = df['year'] - 78       # linear trend
df['t2'] = df['t'] ** 2         # Quadratic trend

 Restringimos la muestra según nuestro interés

In [53]:
df = df[(df['year'] >= 79) & (df['year'] <= 95) & (df['state'] != 98)]

Agregamos más variables demográficas y modificamos la variable de unión

In [54]:
# Generamos las variables demográficas agregadas
df['clp'] = df['clg'] + df['gtc']
df['a1624'] = df[['m1619', 'm2024', 'f1619', 'f2024']].sum(axis=1)
df['a2554'] = df['m2554'] + df['f2554']
df['a55up'] = df[['m5564', 'm65up', 'f5564', 'f65up']].sum(axis=1)
df['fem'] = df[['f1619', 'f2024', 'f2554', 'f5564', 'f65up']].sum(axis=1)
df['white'] = df['rs_wm'] + df['rs_wf']
df['black'] = df['rs_bm'] + df['rs_bf']
df['other'] = df['rs_om'] + df['rs_of']
df['married'] = df['marfem'] + df['marmale']

# Modificamos la variable de union
df.loc[df['year'].isin([79, 81]), 'unmem'] = np.nan  # Excluimos 1979 y 1981
df['unmem'] = df['unmem'] * 100   # A porcentaje

Antes de realizar las regresiones, adecuamos la base de dato a un multindex para poder regresionar

In [55]:
df = df.set_index(['state', 'year'])
df

ue    cpsths    tempue     hrwage      wkwage     lnhrw  \
state year                                                                  
11.0  79.0  0.077437  0.000000  0.001249   4.969775  194.019363  1.494992   
      80.0  0.080174  0.000325  0.000000   5.438186  211.702667  1.582148   
      81.0  0.070858  0.000780  0.000000   5.825861  228.081894  1.659217   
      82.0  0.084640  0.001092  0.000000   6.280896  242.448547  1.724446   
      83.0  0.091918  0.001789  0.000404   6.577292  258.201538  1.768105   
...              ...       ...       ...        ...         ...       ...   
95.0  91.0  0.028089  0.003349  0.000000  12.482441  498.799286  2.372595   
      92.0  0.046817  0.003086  0.000000  12.795998  504.817474  2.406605   
      93.0  0.043933  0.003606  0.000000  13.194592  520.331238  2.437165   
      94.0  0.060942  0.003455  0.000000  13.323984  520.674988  2.436631   
      95.0  0.057788  0.002229  0.000000  13.050425  507.867493  2.417453   

               lnwkw     lnftw       hsd       hsg  ...     t2       clp  \
state year                                          ...                    
11.0  79.0  5.035960  5.310332  0.253758  0.422910  ...    1.0  0.162131   
      80.0  5.143550  5.381139  0.245879  0.426097  ...    4.0  0.155715   
      81.0  5.217398  5.471805  0.241819  0.423842  ...    9.0  0.155863   
      82.0  5.275512  5.527994  0.220420  0.427589  ...   16.0  0.168504   
      83.0  5.311575  5.597443  0.189732  0.436904  ...   25.0  0.176277   
...              ...       ...       ...       ...  ...    ...       ...   
95.0  91.0  5.985239  6.169759  0.092416  0.347033  ...  169.0  0.289614   
      92.0  5.999764  6.214530  0.083364  0.347915  ...  196.0  0.280986   
      93.0  6.041272  6.222130  0.076595  0.348953  ...  225.0  0.271185   
      94.0  6.012619  6.188493  0.087524  0.339075  ...  256.0  0.245760   
      95.0  5.979689  6.177753  0.083665  0.325533  ...  289.0  0.261959   

               a1624     a2554     a55up       fem     white     black  \
state year                                                               
11.0  79.0  0.246696  0.605539  0.147765  0.424343  0.992131  0.002003   
      80.0  0.238558  0.617534  0.143908  0.427401  0.989318  0.002671   
      81.0  0.242791  0.622122  0.135087  0.421965  0.992249  0.002859   
      82.0  0.224009  0.646418  0.129572  0.429675  0.991188  0.002615   
      83.0  0.221283  0.640045  0.138671  0.441355  0.988654  0.002282   
...              ...       ...       ...       ...       ...       ...   
95.0  91.0  0.131415  0.714378  0.154207  0.471808  0.346777  0.015227   
      92.0  0.141886  0.720312  0.137802  0.481401  0.348110  0.016678   
      93.0  0.125163  0.736133  0.138704  0.472990  0.329735  0.020363   
      94.0  0.132026  0.729509  0.138465  0.495596  0.345330  0.022824   
      95.0  0.149073  0.722274  0.128653  0.495241  0.369253  0.021390   

               other   married  
state year                      
11.0  79.0  0.005866  0.665771  
      80.0  0.008012  0.649958  
      81.0  0.004893  0.653300  
      82.0  0.006197  0.662008  
      83.0  0.009064  0.654446  
...              ...       ...  
95.0  91.0  0.637996  0.593802  
      92.0  0.635212  0.589891  
      93.0  0.649902  0.617761  
      94.0  0.631845  0.588175  
      95.0  0.609357  0.608256  

[850 rows x 613 columns]

Realizamos la regresión DID

In [56]:
# Creamos year y state como columnas temporales para la regresión
df['state'] = df.index.get_level_values('state')
df['year'] = df.index.get_level_values('year')
# Función para aplicar una regresión de efectos fijos
formula = formula = "lnths ~ 1 + lnemp + admico_2 + admico_1 + admico0 + admico1 + admico2 + admico3 + mico4 + \
            admppa_2 + admppa_1 + admppa0 + admppa1 + admppa2 + admppa3 + mppa4 + \
            admgfa_2 + admgfa_1 + admgfa0 + admgfa1 + admgfa2 + admgfa3 + mgfa4 + \
            C(year) + C(state) + C(state):t + EntityEffects" # Formula de la regresión
model = PanelOLS.from_formula(formula, data=df, check_rank=False, drop_absorbed=True) # Aplicamos al conjunto de datos y usamos los pesos según población
result = model.fit(cov_type="clustered", cluster_entity=True,)   # La clusterización por grupos 
print(result)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lnths   R-squared:                        0.9475
Estimator:                   PanelOLS   R-squared (Between):              0.8055
No. Observations:                 850   R-squared (Within):               0.9475
Date:                Thu, Oct 31 2024   R-squared (Overall):              0.8354
Time:                        14:24:13   Log-likelihood                    252.82
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      147.88
Entities:                          50   P-value                           0.0000
Avg Obs:                       17.000   Distribution:                  F(87,713)
Min Obs:                       17.000                                           
Max Obs:                       17.000   F-statistic (robust):          1.007e+16
                            